In [ ]:
from tensorflow.python.keras import backend
import tensorflow as tf
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Model
from tensorboard.plugins.hparams import api as hp
import numpy as np
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tb_imgs_utilities import image_grid, plot_to_image, confusion_matrix, plot_confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from tensorflow.python.keras.engine import data_adapter
from sklearn.metrics import roc_auc_score

In [ ]:
# CONSTANTS
N_CLASS = 10
enc = OneHotEncoder(handle_unknown='ignore')
cl = np.arange(N_CLASS).reshape((N_CLASS, 1))
enc.fit(cl)
log_dir = 'logs/InceptionResnet_best'  # './logs/'
checkpoint_filepath = 'checkpoint_InceptionResnet_best/'  # './checkpoint/'

# PREPARE DATA
DATA_BASE_FOLDER = '/kaggle/input/ml-project-2020-dataset/'
x_train = np.load(os.path.join(DATA_BASE_FOLDER, 'train.npy'))
x_valid = np.load(os.path.join(DATA_BASE_FOLDER, 'validation.npy'))
x_test = np.load(os.path.join(DATA_BASE_FOLDER, 'test.npy'))
y_train = pd.read_csv(os.path.join(DATA_BASE_FOLDER, 'train.csv'))['class'].values
y_valid = pd.read_csv(os.path.join(DATA_BASE_FOLDER, 'validation.csv'))['class'].values

y_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
x_train = x_train.reshape(x_train.shape[0], 28, 28)  # reconstruct images
x_valid = x_valid.reshape(x_valid.shape[0], 28, 28)
x_test = x_test.reshape(x_test.shape[0], 28, 28)
x_train = tf.expand_dims(x_train, axis=-1)
x_valid = tf.expand_dims(x_valid, axis=-1)
x_test = tf.expand_dims(x_test, axis=-1)

In [ ]:
from test_dataset import InceptionResNetV2Fashion, CMTensorBoardCallback
#,conv2d_bn, inception_resnet_block

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label
BATCH_SIZE = 32

# Training two models selected
(training one model by step, since training requires much time......)
(more 'clean' codes to reduce training time)

## The smaller model

In [ ]:
dir_name = ' ' + 'small'
print('HP ---> ', dir_name)
file_writer = tf.summary.create_file_writer(log_dir + dir_name)
tb_cm_callback = CMTensorBoardCallback(file_writer,
                                        log_dir=(log_dir + dir_name))
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath + dir_name,
                                                         save_weights_only=True,
                                                         monitor='val_accuracy',
                                                         mode='max',
                                                         save_best_only=True)
early_callback = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy', min_delta=0, patience=6, verbose=1,
    mode='max'
)
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', factor=0.2, mode = 'max',
                              patience=3, min_lr=0.0001, verbose=1)

model = InceptionResNetV2Fashion((28, 28, 1), N_CLASS,
                                 2,
                                 0.5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

metric = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric],
              run_eagerly=False)

# CREATE DATASET
x_train_flip = tf.image.random_flip_left_right(x_train)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_flip, y_train))
valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))

train_dataset = train_dataset.map(scale).cache().shuffle(10000).batch(BATCH_SIZE).prefetch(3)
valid_dataset = valid_dataset.map(scale).batch(BATCH_SIZE*10) #scale function MANDATORY

print('*************** START TRAINING *******************')
model.fit(train_dataset,
          validation_data=valid_dataset,
          epochs=10,
          callbacks=[tb_cm_callback, checkpoint_callback, early_callback, reduce_lr_callback],
          verbose=2)

### Confusion Matrix plot

In [ ]:
# file_writer = tf.summary.create_file_writer(log_dir + dir_name)
# 
# # CMTensorBoardCallback codes
# # Use the model to predict the values from the validation dataset.
# test_pred_raw = model.predict(x_valid)  # for large input
# # AUROC print
# y_pred = np.squeeze(test_pred_raw)
# y_ohe = np.squeeze(y_valid)
# y_ohe = np.expand_dims(y_ohe, axis=1)
# y_ohe = enc.transform(y_ohe).toarray()
# r_sum = 0
# for c in range(N_CLASS):
#     try:
#         r = roc_auc_score(y_ohe[:, c], y_pred[:, c])
#         print('Label: ', y_labels[c], ', ', 'AUROC one vs rest = ', r)
#         r_sum += r
#     except ValueError:
#         print('Label: ', y_labels[c], ', ', 'AUROC one vs rest = None!')
# mean_auroc = r_sum / N_CLASS
# print('MEAN AUROC: ', mean_auroc)
# 
# test_pred = np.argmax(test_pred_raw, axis=1)
# # Calculate the confusion matrix.
# cm = confusion_matrix(y_valid, test_pred)
# # Log the confusion matrix as an image summary.
# figure = plot_confusion_matrix(cm, y_labels)
# cm_image = plot_to_image(figure)
# # Log the confusion matrix as an image summary.
# with file_writer.as_default():
#     tf.summary.image("End training small model: Confusion Matrix", cm_image,
#                      description='Smaller InceptionResNetV2Fashion: initial_lr=0.001(ReduceLROnPlateau), Adam optimizer and drop_rate=0.5)


## The bigger model

In [ ]:
# dir_name = ' ' + 'big'
# print('HP ---> ', dir_name)
# file_writer = tf.summary.create_file_writer(log_dir + dir_name)
# tb_cm_callback = CMTensorBoardCallback(file_writer,
#                                         log_dir=(log_dir + dir_name))
# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath + dir_name,
#                                                          save_weights_only=True,
#                                                          monitor='val_accuracy',
#                                                          mode='max',
#                                                          save_best_only=True)
# early_callback = tf.keras.callbacks.EarlyStopping(
#     monitor='accuracy', min_delta=0, patience=6, verbose=1,
#     mode='max'
# )
# reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', factor=0.2, mode = 'max',
#                               patience=3, min_lr=0.0001, verbose=1)
# 
# model = InceptionResNetV2Fashion((28, 28, 1), N_CLASS,
#                                  1,
#                                  0.5)
# loss = tf.keras.losses.SparseCategoricalCrossentropy()
# 
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# 
# metric = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')
# 
# model.compile(optimizer=optimizer, loss=loss, metrics=[metric],
#               run_eagerly=False)
# 
# # CREATE DATASET
# x_train_flip = tf.image.random_flip_left_right(x_train)
# train_dataset = tf.data.Dataset.from_tensor_slices((x_train_flip, y_train))
# valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
# 
# train_dataset = train_dataset.map(scale).cache().shuffle(10000).batch(BATCH_SIZE).prefetch(2)
# valid_dataset = valid_dataset.map(scale).batch(BATCH_SIZE*10) #scale function MANDATORY
# 
# print('*************** START TRAINING *******************')
# model.fit(train_dataset,
#           validation_data=valid_dataset,
#           epochs=10,
#           callbacks=[tb_cm_callback, checkpoint_callback, early_callback, reduce_lr_callback],
#           verbose=2)


### Confusion Matrix plot

In [ ]:
# file_writer = tf.summary.create_file_writer(log_dir + dir_name)
# 
# # CMTensorBoardCallback codes
# # Use the model to predict the values from the validation dataset.
# test_pred_raw = model.predict(x_valid)  # for large input
# # AUROC print
# y_pred = np.squeeze(test_pred_raw)
# y_ohe = np.squeeze(y_valid)
# y_ohe = np.expand_dims(y_ohe, axis=1)
# y_ohe = enc.transform(y_ohe).toarray()
# r_sum = 0
# for c in range(N_CLASS):
#     try:
#         r = roc_auc_score(y_ohe[:, c], y_pred[:, c])
#         print('Label: ', y_labels[c], ', ', 'AUROC one vs rest = ', r)
#         r_sum += r
#     except ValueError:
#         print('Label: ', y_labels[c], ', ', 'AUROC one vs rest = None!')
# mean_auroc = r_sum / N_CLASS
# print('MEAN AUROC: ', mean_auroc)
# 
# test_pred = np.argmax(test_pred_raw, axis=1)
# # Calculate the confusion matrix.
# cm = confusion_matrix(y_valid, test_pred)
# # Log the confusion matrix as an image summary.
# figure = plot_confusion_matrix(cm, y_labels)
# cm_image = plot_to_image(figure)
# # Log the confusion matrix as an image summary.
# with file_writer.as_default():
#     tf.summary.image("End training big model: Confusion Matrix", cm_image,
#                      description='Bigger InceptionResNetV2Fashion: initial_lr=0.001(ReduceLROnPlateau), Adam optimizer and drop_rate=0.5)

